<a href="https://colab.research.google.com/github/rithikkulkarni/simstudent-data-preprocessing/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

file_path = 'https://github.com/rithikkulkarni/simstudent-data-preprocessing/raw/main/Algebra2017May(YesPrep)-APLUS-SUBSET.xlsx'

df = pd.read_excel(file_path, sheet_name="Algebra2017May(YesPrep)-APLUS", engine="openpyxl")
print(df.shape)

(140943, 62)


In [ ]:
unique_students = df['Anon Student Id'].nunique()
print(f'# of Unique Students: {unique_students}')

# of Unique Students: 266


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140943 entries, 0 to 140942
Data columns (total 62 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   Row                          140943 non-null  int64         
 1   Sample Name                  140943 non-null  object        
 2   Transaction Id               140943 non-null  object        
 3   Anon Student Id              140943 non-null  object        
 4   Session Id                   140943 non-null  object        
 5   Time                         140943 non-null  datetime64[ns]
 6   Time Zone                    140943 non-null  object        
 7   Duration (sec)               140943 non-null  object        
 8   Student Response Type        140943 non-null  object        
 9   Student Response Subtype     63907 non-null   object        
 10  Tutor Response Type          136474 non-null  object        
 11  Tutor Response Subtype    

In [ ]:
### REMOVAL OF UNWANTED COLUMNS (RUN ONLY ONCE)
df = df.drop([
  'Feedback Classification',
  'Total Num Hints',
  'Condition Type',
  'KC Category (Single-KC)',
  'KC Category (Unique-step)',
  'Unnamed: 60',
  'Unnamed: 61',
  'Row',
  'Sample Name',
  'Transaction Id',
  'School',
  'Time Zone',
  'Time',
  'Tutor Response Type',
  'Tutor Response Subtype',
  'Level (Unit)',
  'Problem Name',
  'Step Name',
  'Selection',
  'Action',
  'KC (Problem)',
  'KC (Step)',
  'KC Category (Step)',
  'KC (ActualSkill)',
  'KC Category (ActualSkill)',
  'KC (Default)',
  'KC Category (Default)',
  'KC (ProblemSubmit)',
  'KC Category (ProblemSubmit)',
  'KC (Single-KC)',
  'KC (Unique-step)',
  'Class',
  'CF (HINT_FOLLOWED)',
  'CF (HINT_SUBJECT)',
  'CF (HINT_TYPE)',
  'CF (INFO)',
  'CF (RESULT)',
  'CF (USERID)',
  'CF (STUDENT_LOG_COUNT)',
  'CF (date)',
  'Participation Day Key',
  'Student Response Subtype',
  'Problem Start Time'
], axis=1)

In [ ]:
# Renaming columns in preparation for dataset export
df = df.rename(columns={
  'Anon Student Id': 'student_id',
  'Session Id': 'session_id',
  'Duration (sec)': 'duration_sec',
  'Student Response Type': 'student_response_type',
  'Problem View': 'problem_view',
  'Attempt At Step': 'attempt_at_step',
  'Is Last Attempt': 'is_last_attempt',
  'Outcome': 'outcome',
  'Input': 'input',
  'Feedback Text': 'feedback_text',
  'Help Level': 'help_level',
  'Condition Name': 'condition_name',
  'KC Category (Problem)': 'problem_category',
  'CF (ACTION)': 'action_explanation',
  'CF (ACTION_TYPE)': 'action',
  'CF (DURATION)': 'duration',
  'CF (STATUS)': 'status',
  'Participation Day': 'participation_day',
  'CF (tool_event_time)': 'tool_event_time'
})

In [ ]:
# Get a visual of the amount of unique values in each column

for col in df.columns:
  print(f'{col}: {df[col].nunique()}')

student_id: 266
session_id: 856
duration_sec: 1100
student_response_type: 2
problem_view: 25
attempt_at_step: 385
is_last_attempt: 2
outcome: 4
input: 1944
feedback_text: 8253
help_level: 4
condition_name: 2
problem_category: 8
action_explanation: 58
action: 12
duration: 2242
status: 3
participation_day: 4
tool_event_time: 140041


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140943 entries, 0 to 140942
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   student_id             140943 non-null  object 
 1   session_id             140943 non-null  object 
 2   duration_sec           140943 non-null  object 
 3   student_response_type  140943 non-null  object 
 4   problem_view           140943 non-null  int64  
 5   attempt_at_step        136474 non-null  float64
 6   is_last_attempt        136474 non-null  float64
 7   outcome                136474 non-null  object 
 8   input                  68954 non-null   object 
 9   feedback_text          20420 non-null   object 
 10  help_level             5806 non-null    float64
 11  condition_name         140943 non-null  object 
 12  problem_category       86862 non-null   object 
 13  action_explanation     138286 non-null  object 
 14  action                 138286 non-nu

In [ ]:
df.to_excel('SimStudent_StudyVII_May2017_Preprocessed_Subset.xlsx', index=False)